# Silver Layer 🥈

## Step 1: Imports

In [ ]:
import pandas as pd
import numpy as np
import string
import datetime


## Step 2: Load the Bronze Layer data

In [ ]:
df = pd.read_parquet('/bronze/bronze_mario.parquet') # read bronze parquet


## Step 3: Generalized Data Validation and Cleaning Functions

In [ ]:
def drop_null_columns(df, threshold=0.2):
    """Remove (drop) mostly-empty columns. Threshold=0.2 → means 20%. → means 20% missing allowed."""
    null_percentage = df.isnull().mean()
    columns_to_keep = null_percentage[null_percentage < threshold].index 

    if len(columns_to_keep) == len(df.columns):
        print('no columns remvoed')
    else:
        print(f'removing: {[c for c in df.columns if c not in columns_to_keep]}')

    return df[columns_to_keep]


In [ ]:
def fill_missing_values(df, columns_defaults: dict):
    """Fill missing values with assigned defaults in columns default dictionary."""
    for column, default_value in columns_defaults.items():
        df[column] = df[column].fillna(default_value)
    return df


In [ ]:
def convert_column_types(df, columns_types: dict):
    """Convert columns to desired data types."""
    try:
        for column, dtype in columns_types.items():
            df[column] = df[column].astype(dtype)
        return df
    except Exception as e:
        print(f'{column} caused an issue')
        raise e
    

In [ ]:
def remove_punctuation(df, columns: list):
    """Remove punctuation from string type columns"""
    for c in columns:
        df.loc[:,c] = df[c].str.replace(r'[^\w\s]', '', regex=True)
    return df


In [ ]:
def clean_spaces(df, columns: list):
    """Remove leading and trailing spaces from specific string columns"""
    for c in columns:
        df.loc[:, c] = df[c].astype(str).str.strip()
    return df


In [ ]:
def check_formats(df, expected_formats: dict):
    """Check the schema of the table to validate formats"""
    incorrect_formats = []
    for column, datatype in df.dtypes.to_dict().items():
        expected_type = expected_formats.get(column)
        if expected_type != datatype:
            incorrect_formats.append((column, datatype, expected_type))

    incorrect_columns = [c[0] for c in incorrect_formats]
    correct_format_count = len([c for c in df.columns if c not in incorrect_columns])
    if incorrect_formats:
        print('Below are incorrect formats')
        print("-"* 50)
        print(f'correct column count: {correct_format_count}')
        return pd.DataFrame(incorrect_formats, columns=['Column', 'Actual', 'Expected'])
    else:
        print('Validation Complete, no discrepancies')
        

In [ ]:
def check_similarity(word1: str, word2: str) -> float:
    '''Check the similarity between two words using Jaccard Similarity'''
    if pd.isna(word1) or pd.isna(word2):
        return 0.0

    word_set1 = set(word1)
    word_set2 = set(word2)

    intersection = word_set1.intersection(word_set2)
    intersection_count = len(intersection)

    total_char_count = len(word_set1.union(word_set2))

    similarity = intersection_count / total_char_count
    return similarity


In [ ]:
def check_misspelling(dataframe: pd.DataFrame, column: str, similarity_threshold: float) -> pd.DataFrame:
    '''
    Input a pandas dataframe, a specific column, and similarity threshold from 0-1 to get all values that are similar
    dataframe: Pandas DataFrame
    column: String representing a column from Dataframe
    similarity_threshold: Float representing values from 0 to 1 or 0 to 100% similarity
    '''

    all_unique_values = list(set(dataframe[column].tolist()))

    similarity_list = []
    for n in range(len(all_unique_values)):
        value1 = all_unique_values[n]
        for n2 in range(n + 1, len(all_unique_values)):
            value2 = all_unique_values[n2]
            similarity = round(check_similarity(value1, value2), 4)
            if similarity >= similarity_threshold:
                similarity_list.append([value1, value2, similarity])
    return pd.DataFrame(similarity_list, columns=['name1', 'name2', 'similarity'])


## Step 4: Apply Validations and Cleaning

In [ ]:
df.head()


In [ ]:
df.isnull().sum()


In [ ]:
df.isnull().mean() * 100


In [ ]:
drop_null_columns(df, threshold=.25)


In [ ]:
fill_missing_values(df, {'Player Name': 'Unknown Player', 'Team': 'Unknown Team', 'Kart Racing Rank': 'Unknown Rank', 'Mushroom Cup Participation': 'Unknown'})


In [ ]:
df.isnull().mean() * 100


In [ ]:
remove_punctuation(df, ['Kart Role'])


In [ ]:
check_misspelling(df, 'Player Name', .8)



In [ ]:
expected_formats = {
    'Player Name': 'string',
    'Team': 'string',
    'Kart Racing Rank': 'string',
    'Mushroom Cup Participation': 'bool',
    'Participation in Battle Mode': 'bool',
    'Platforming Rank': 'string', 
    'Boss Battle Rank': 'string',
    'Power-Ups Owned': 'string', 
    'Kart Role': 'string',
    'Power-Ups Used': 'int32',
    'Team Points': 'int32',
    'Levels Completed': 'int32',
    'Lives Lost': 'int32', 
    'Times Hit by Enemies': 'int32',
    'Vehicle Type': 'string',
    'World': 'string', 
    'Coins Spent in Toad Town': 'int32',
    'Companion': 'string', 
    'Primary Game': 'string'
    }


In [ ]:
check_formats(df, expected_formats=expected_formats)


In [ ]:
convert_column_types(df, columns_types=expected_formats)


In [ ]:
mario_misspelling = check_misspelling(df, 'Player Name', 0.8)
mario_misspelling = mario_misspelling[
    (mario_misspelling['name1'].str.lower() == 'mario') |
    (mario_misspelling['name2'].str.lower() == 'mario')
]

mario_misspelling

In [ ]:
valid_names = [
    'Mario', 'Luigi', 'Peach', 'Daisy', 'Yoshi', 'Toad', 'Toadette', 
    'Rosalina', 'Wario', 'Waluigi', 'Bowser', 'Bowser Jr.'
]
rename_values = check_misspelling(df, 'Player Name', 0.8)
rename_values = rename_values[
    (rename_values['name1'].isin(valid_names)) | (rename_values['name2'].isin(valid_names))
]

rename_values['typo'] = rename_values.apply(
    lambda r: r['name2'] if r['name1'] in valid_names else r['name1'], axis=1)
rename_values['valid'] = rename_values.apply(
    lambda r: r['name1'] if r['name1'] in valid_names else r['name2'], axis=1)

rename_values = rename_values.sort_values('similarity', ascending=False).drop_duplicates('typo')

renaming = dict(zip(rename_values['typo'], rename_values['valid']))

df['Player Name'] = df['Player Name'].replace(renaming)
sorted(df['Player Name'].unique())



In [ ]:
cols_to_clean = ['Vehicle Type', 'World', 'Primary Game']

cleaned = clean_spaces(df, cols_to_clean)

cleaned[['Vehicle Type', 'World', 'Primary Game']].head()


In [ ]:
cols_to_title = ['Team', 'Companion', 'World']

for col in cols_to_title:
    df.loc[:, col] = df[col].astype(str).str.title()


In [ ]:
df


In [ ]:
df.info()

In [ ]:
check_formats(df, expected_formats=expected_formats)


## Step 5: Save Silver Layer

In [ ]:
df.to_parquet('/silver/silver_mario.parquet', index = False) 
print("✅ Cleaned data saved to Silver layer")